# How to think about APIs ?

this is a pseudo-code example:

APIs are sort of like like functions living online, eg.

`
url = f"http://www.omdbapi.com/?query=StarWars&year=1993"
`

is like calling an online function

`
def search_movies(query, year):
    # some code
`

like this:

`
search_movies("StarWars", 1993)
`

so you can create urls, as if you were passing variables into an online function

`
query = "Star Wars"
page = 2
url = f"http://www.omdbapi.com/?s={query}&page={page}"
`

then when you 'request' the data, your online function is asked for its results

`
response_dictionary = requests.request("GET", url).json()
`

# How to split code into functions (and test it)

this example is from the solution to lab 11, task 13:

In [3]:
opening_hours = [{'day':'Monday', 'open': 9, 'close':18},
                {'day':'Tuesday', 'open': 12, 'close':18},
                {'day':'Wednesday'},
                {'day':'Thursday', 'open': 9, 'close':18},
                {'day':'Friday', 'open': 9, 'close':21},
                {'day':'Saturday', 'open': 12, 'close':18},
                {'day':'Sunday', 'open': 12, 'close':18}]

In [4]:
def top_part():
    result_so_far = ""
    welcome_message = 'WELCOME'
    result_so_far += f"{welcome_message:*^30}\n"
    # welcome surrounded by 30 *
    return result_so_far

In [5]:
assert top_part() == "***********WELCOME************\n"

In [6]:
def bottom_part():
    result_so_far = ""
    result_so_far += f"{'':*^30}\n" 
    # empty string surrounded by 30 *
    return result_so_far

In [7]:
assert bottom_part() == "******************************\n"

In [8]:
def day_part(day_info):
    opening_hours_text = ""
    if day_info.get('open') == None or day_info.get('close') == None:
        opening_hours_text = 'Closed'
    else:
        opening_hours_text += f"{day_info.get('open'):2} till {day_info.get('close'):2}"
    return f"* {day_info['day']:.<10}{opening_hours_text:.^16} *\n"

In [9]:
assert day_part(opening_hours[0]) == "* Monday....... 9 till 18... *\n"
assert day_part(opening_hours[2]) == "* Wednesday......Closed..... *\n"

In [10]:
def create_sign(week_data):
    final_sign_so_far = ""
    final_sign_so_far += top_part()
    for day_data in week_data:
        final_sign_so_far += day_part(day_data)
    final_sign_so_far += bottom_part()
    return final_sign_so_far

In [11]:
print(create_sign(opening_hours))

***********WELCOME************
* Monday....... 9 till 18... *
* Tuesday......12 till 18... *
* Wednesday......Closed..... *
* Thursday..... 9 till 18... *
* Friday....... 9 till 21... *
* Saturday.....12 till 18... *
* Sunday.......12 till 18... *
******************************



# complicated example of splitting code

you could put all the code into one cell, but that will get quickly messy, for example

In [12]:
# don't just say

import requests
import pprint as pp

your_api_key = "dd14dc5f" #use mine, or get your own from the above website
page = 1 # each page has 10 results, so page 1 has results 1-10, page 2 has results 11-20, etc
# how many results are there in total? look at the 'totalResults' attribute in response dictionary
query = "star wars"
result_type="movie" # read the omdb website to see what other options there are
year = "" #leave empty to get all results all years


# get all moviue info
url = f"http://www.omdbapi.com/?s={query}&page={page}&apikey={your_api_key}&type={result_type}&y={year}"
response = requests.request("GET", url)
star_wars_movies = response.json()['Search']

# just keep ids
star_wars_movies_ids = [movie['imdbID']
                        for movie in star_wars_movies]

# get details and print them
for movie_id in star_wars_movies_ids:
    url = f"http://www.omdbapi.com/?i={movie_id}&apikey={your_api_key}"
    response_one_movie = requests.request("GET", url).json()
    actors_string = response_one_movie["Actors"]
    actors_list = actors_string.split(", ") 
    print(f"First actor in {response_one_movie['Title']} is {actors_list[0]}")


First actor in Star Wars: Episode IV - A New Hope is Mark Hamill
First actor in Star Wars: Episode V - The Empire Strikes Back is Mark Hamill
First actor in Star Wars: Episode VI - Return of the Jedi is Mark Hamill
First actor in Star Wars: Episode VII - The Force Awakens is Harrison Ford
First actor in Star Wars: Episode I - The Phantom Menace is Liam Neeson
First actor in Star Wars: Episode III - Revenge of the Sith is Ewan McGregor
First actor in Star Wars: Episode II - Attack of the Clones is Ewan McGregor
First actor in Star Wars: Episode VIII - The Last Jedi is Mark Hamill
First actor in Rogue One: A Star Wars Story is Felicity Jones
First actor in Star Wars: Episode IX - The Rise of Skywalker is Carrie Fisher


but you could also split your code into functions, like this

In [13]:

import requests
import pprint as pp


def get_ids_of_movies_named(movie_name):
    result_type="movie"
    your_api_key = "dd14dc5f" 

    url = f"http://www.omdbapi.com/?s={movie_name}&page={page}&apikey={your_api_key}&type={result_type}"
    response = requests.request("GET", url)
    movies = response.json()['Search']
    movies_ids = [movie['imdbID']
                        for movie in movies]
    return movies_ids

def get_all_details_of_movies_with_ids(movie_ids):
    your_api_key = "dd14dc5f" 
    movie_detail_strings = []
    for movie_id in movie_ids:
        url = f"http://www.omdbapi.com/?i={movie_id}&apikey={your_api_key}"
        response_one_movie = requests.request("GET", url).json()
        info_string = info_string_from_movie_details(response_one_movie) 
        movie_detail_strings.append(  info_string )
    return movie_detail_strings

def info_string_from_movie_details(movie_details):
    actors_list = movie_details["Actors"].split(", ") 
    return f"First actor in {movie_details['Title']} is {actors_list[0]}"

def print_starwars_details():
    movie_ids = get_ids_of_movies_named("star wars")
    movies_details_list = get_all_details_of_movies_with_ids(movie_ids)
    for movie_details_string in movies_details_list:
        print(movie_details_string)
        
print_starwars_details()

First actor in Star Wars: Episode IV - A New Hope is Mark Hamill
First actor in Star Wars: Episode V - The Empire Strikes Back is Mark Hamill
First actor in Star Wars: Episode VI - Return of the Jedi is Mark Hamill
First actor in Star Wars: Episode VII - The Force Awakens is Harrison Ford
First actor in Star Wars: Episode I - The Phantom Menace is Liam Neeson
First actor in Star Wars: Episode III - Revenge of the Sith is Ewan McGregor
First actor in Star Wars: Episode II - Attack of the Clones is Ewan McGregor
First actor in Star Wars: Episode VIII - The Last Jedi is Mark Hamill
First actor in Rogue One: A Star Wars Story is Felicity Jones
First actor in Star Wars: Episode IX - The Rise of Skywalker is Carrie Fisher
